In [1]:
import pickle
import pandas as pd
import numpy as np

/Users/alishakhan/opt/miniconda3/envs/alishadev/lib/python3.9/site-packages/pandas/compat/_optional.py:149: UserWarning: Pandas requires version '1.3.1' or newer of 'bottleneck' (version '1.2.1' currently installed).
  warnings.warn(msg, UserWarning)


In [58]:
# Loading Datasets
with open('/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/feature_extraction/train_df.pickle', 'rb') as f:
    train_df = pickle.load(f)

with open('/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/feature_extraction/test_df.pickle', 'rb') as f:
    test_df = pickle.load(f)

with open('/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/feature_extraction/val_df.pickle', 'rb') as f:
    val_df = pickle.load(f)

#combining test and val
test_df = pd.concat([test_df, val_df])

# CLEANING DATASETS we dont care about wall(2), background(0), outdoor(1), garage (10), undefined (11), railing (8)
train_df = train_df[~train_df['Room'].isin([0, 1, 2, 10, 11, 8])]
train_df = train_df.loc[:, train_df.columns != 'No Icon']

test_df = test_df[~test_df['Room'].isin([0, 1, 2, 10, 11, 8])]
test_df = test_df.loc[:, test_df.columns != 'No Icon']

# Train test
X_train = train_df[['Window', 'Door', 'Closet', 'Electrical Applience', 'Toilet', 'Sink', 'Sauna Bench', 'Fire Place', 'Bathtub', 'Chimney']]
y_train = train_df['Room']

X_test = test_df[['Window', 'Door', 'Closet', 'Electrical Applience', 'Toilet', 'Sink', 'Sauna Bench', 'Fire Place', 'Bathtub', 'Chimney']]
y_test = test_df['Room']

# Classification Algorithms

## Multinomial NB

In [59]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


# Define the MNB classifier model
mnb = MultinomialNB()

# Train the model on the training set
mnb.fit(X_train, y_train)

# Predict the target variable on the testing set
y_pred = mnb.predict(X_test)

# Evaluate the model performance
print(classification_report(y_test, y_pred))

# Compute the accuracy score
accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy score
print("Accuracy:", accuracy)


              precision    recall  f1-score   support

           3       0.81      0.94      0.87       722
           4       0.71      0.25      0.37       671
           5       0.36      0.96      0.53      1257
           6       0.99      0.81      0.89      1159
           7       0.45      0.04      0.08       922
           9       0.45      0.03      0.05       758

    accuracy                           0.56      5489
   macro avg       0.63      0.51      0.46      5489
weighted avg       0.62      0.56      0.49      5489

Accuracy: 0.5563854982692658


In [60]:
# Get the predicted probabilities on the testing set
probs = pd.DataFrame(mnb.predict_proba(X_test), columns=mnb.classes_)

# Combine the actual and predicted values with the probabilities
mnb_predictions = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred}).reset_index(drop=True)
probs = pd.concat([mnb_predictions, probs], axis=1)

# Get the maximum probability for each instance
probs_max = probs.iloc[:, 2:].max(axis=1)
probs_max = pd.concat([probs.iloc[:, 0:2], probs_max], axis=1)
probs_max.columns.values[2] = "probability"

# Vary the probability threshold and compute the resulting accuracy
for i in np.arange(0, 1, 0.01):
    all_rows = probs_max[probs_max['probability'] >= i]
    correct = all_rows.query('Actual == Predicted')
    ratio = len(correct) / len(all_rows)
    if ratio >= 0.9:
        print("Ratio:", ratio)
        print("Threshold:", i)
        break

# Compute the accuracy for the threshold of 0.51
accuracy_high = len(probs_max.query(f'probability >= {i}').query('Actual == Predicted')) / len(probs_max.query(f'probability >= {i}'))
print(f"Accuracy for threshold >= {i}:", accuracy_high)

# Compute the accuracy for the threshold of 0.5
accuracy_low = len(probs_max.query(f'probability < {i}').query('Actual == Predicted')) / len(probs_max.query(f'probability < {i}'))
print(f"Accuracy for threshold < {i}:", accuracy_low)


Ratio: 0.9002991026919243
Threshold: 0.48
Accuracy for threshold >= 0.48: 0.9002991026919243
Accuracy for threshold < 0.48: 0.35831180017226527


In [61]:
filename = 'room_classification_model_1.pkl'
with open(filename, 'wb') as file:
    pickle.dump(mnb, file)

## Random Forest Classifier

In [40]:
from sklearn.ensemble import RandomForestClassifier

# Define the random forest classifier model
rfc = RandomForestClassifier()

# Train the model on the training set
rfc.fit(X_train, y_train)

# Predict the target variable on the testing set
y_pred = rfc.predict(X_test)

# Evaluate the model performance
print(classification_report(y_test, y_pred))

# Compute the accuracy score
accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy score
print("Accuracy:", accuracy)

              precision    recall  f1-score   support

           3       0.95      0.91      0.93       722
           4       0.33      0.88      0.47       671
           5       0.51      0.63      0.56      1257
           6       0.92      0.86      0.89      1159
           7       0.44      0.15      0.23       922
           9       0.39      0.02      0.03       758

    accuracy                           0.58      5489
   macro avg       0.59      0.57      0.52      5489
weighted avg       0.60      0.58      0.54      5489

Accuracy: 0.5797048642740026


## KNN

In [31]:
from sklearn.neighbors import KNeighborsClassifier

# Define the k-nearest neighbors classifier model
knn = KNeighborsClassifier()

# Train the model on the training set
knn.fit(X_train, y_train)

# Predict the target variable on the testing set
y_pred = knn.predict(X_test)

# Evaluate the model performance
print(classification_report(y_test, y_pred))

# Compute the accuracy score
accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy score
print("Accuracy:", accuracy)

              precision    recall  f1-score   support

           3       0.95      0.91      0.93       722
           4       0.81      0.24      0.37       671
           5       0.50      0.64      0.56      1257
           6       0.98      0.83      0.90      1159
           7       0.41      0.16      0.23       922
           9       0.24      0.52      0.33       758

    accuracy                           0.57      5489
   macro avg       0.65      0.55      0.55      5489
weighted avg       0.65      0.57      0.57      5489

Accuracy: 0.5689560940061942


/Users/alishakhan/opt/miniconda3/envs/alishadev/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


## Multiclass Logistic Regression

In [33]:
from sklearn.linear_model import LogisticRegression

# Define the logistic regression classifier model
lr = LogisticRegression(max_iter=1000)

# Train the model on the training set
lr.fit(X_train, y_train)

# Predict the target variable on the testing set
y_pred = lr.predict(X_test)

# Evaluate the model performance
print(classification_report(y_test, y_pred))

# Compute the accuracy score
accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy score
print("Accuracy:", accuracy)

              precision    recall  f1-score   support

           3       0.96      0.91      0.93       722
           4       0.65      0.28      0.39       671
           5       0.36      0.97      0.53      1257
           6       0.96      0.84      0.90      1159
           7       0.52      0.09      0.15       922
           9       0.50      0.01      0.01       758

    accuracy                           0.57      5489
   macro avg       0.66      0.52      0.49      5489
weighted avg       0.65      0.57      0.51      5489

Accuracy: 0.5689560940061942
